In [ ]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from sklearn.metrics import plot_confusion_matrix

from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.metrics import plot_confusion_matrix
from pyriemann.estimation import XdawnCovariances
from pyriemann.tangentspace import TangentSpace

from sklearn.ensemble import RandomForestClassifier
from time import time


folder = "../Collected_Data/FormattedEEG/"
types = ["control", "left_blink", "right_blink"]
paths = np.array([["left_blink_var1", 1],["left_blink_var2", 1], ["right_blink_var1", 2], ["right_blink_var2", 2], ["control_var1", 0], ["control_var2", 0]])

# Read in csv files
def read_csvs(folder, types, paths):
    output_x = []
    output_y = []
    
    for path in paths:
        for i in range(1, 11):
            if path[0].find("control") != -1 and i == 6:
                break;
            cur_path = folder + types[int(path[1])] + "/" + path[0] + "_" + str(i) + "_" + "formatted.csv"
            input_data = pd.read_csv(cur_path, header = 0, usecols=[1, 2, 3, 4])
            X = input_data.to_numpy()[250:1000]
            X = X.T
            y = path[1]
            output_x.append(X)
            output_y.append(float(y))
    
    output_x = np.array(output_x) 
    output_y = np.array(output_y)
    return output_x, output_y

X, y = read_csvs(folder, types, paths)


cv = KFold(n_splits=4, shuffle=True)

start = time()
clf = make_pipeline(XdawnCovariances(estimator='oas'), TangentSpace(metric = "riemann"), RandomForestClassifier(n_estimators=1000, n_jobs = 8))

pred = np.zeros(len(y))
print(len(y))

for train_idx, test_idx in cv.split(X):
    y_train, y_test = y[train_idx], y[test_idx]
    clf.fit(X[train_idx], y_train)
    pred[test_idx] = clf.predict(X[test_idx])

acc = np.mean(pred == y)
print("Classification accuracy: %f " % (acc))
end = time()
print("Time taken: " + str(end - start))
plot_confusion_matrix(clf, X, y, display_labels=types)  
plt.show()  

In [ ]:
from time import time

folder = "../Collected_Data/FormattedEEG/"
types = ["control", "left_blink", "right_blink"]
paths_orig = np.array([["left_blink_var1", 1],["left_blink_var2", 1], ["right_blink_var1", 2], ["right_blink_var2", 2], ["control_var1", 0], ["control_var2", 0]])
noleftvar1 = np.array([["left_blink_var2", 1], ["right_blink_var1", 2], ["right_blink_var2", 2], ["control_var1", 0], ["control_var2", 0]])
noleftvar2 = np.array([["left_blink_var1", 1], ["right_blink_var1", 2], ["right_blink_var2", 2], ["control_var1", 0], ["control_var2", 0]])
norightvar1 = np.array([["left_blink_var1", 1],["left_blink_var2", 1], ["right_blink_var2", 2], ["control_var1", 0], ["control_var2", 0]])
norightvar2 = np.array([["left_blink_var1", 1],["left_blink_var2", 1], ["right_blink_var1", 2], ["control_var1", 0], ["control_var2", 0]])
nocontrol1 = np.array([["left_blink_var1", 1],["left_blink_var2", 1], ["right_blink_var1", 2], ["right_blink_var2", 2], ["control_var2", 0]])
nocontrol2 = np.array([["left_blink_var1", 1],["left_blink_var2", 1], ["right_blink_var1", 2], ["right_blink_var2", 2], ["control_var1", 0]])

paths_list = [[paths_orig, "original"], [noleftvar1, "no left blink var1"], [noleftvar2, "no left blink var2"], [norightvar1, "no right var1"], [norightvar2, "no right var2"], [nocontrol1, "no control var 1"], [nocontrol2, "no control var 2"]]
for paths in paths_list:
    X, y = read_csvs(folder, types, paths[0])


    cv = KFold(n_splits=4, shuffle=True)

    clf = make_pipeline(XdawnCovariances(estimator='oas'), TangentSpace(metric = "riemann"), RandomForestClassifier(n_estimators=500, n_jobs=4))

    pred = np.zeros(len(y))
    print("Test: " + paths[1])
    print("Test size:" + str(len(y)))

    for train_idx, test_idx in cv.split(X):
        y_train, y_test = y[train_idx], y[test_idx]
        clf.fit(X[train_idx], y_train)
        pred[test_idx] = clf.predict(X[test_idx])

    count = 0
    wrong_labelled = []
    for j in range(len(pred)):
        if pred[j] == y[j]:
            count += 1
        else:
            wrong_labelled.append(j)
    print(wrong_labelled)
    acc = count / len(pred)
    print("Classification accuracy: %f \n" % (acc))
    plot_confusion_matrix(clf, X, y, display_labels=types)  
    plt.show()  

In [74]:
from queue import Queue
from pylsl import StreamInlet, resolve_stream
import matplotlib.pyplot as plt
import numpy as np

import keyboard
import time

def stream_sliding(buffer, chunk, load_data_ML):
    buffer[:-1] = buffer[1:]
    buffer[-1:] = [chunk]

    return buffer

def main():
    buffer = np.full((750, 4), -10000)
    load_data_ML = True
    # first resolve an EEG stream on the lab network
    print("looking for an EEG stream...")
    streams = resolve_stream('type', 'EEG')

    # create a new inlet to read from the stream
    inlet = StreamInlet(streams[0])

    while True:
        if keyboard.is_pressed("q"):
            print("exiting program")
            break
        chunk,_ = inlet.pull_sample()
        chunk = np.array(chunk)[0: 4]
        #print(chunk)

        buffer = stream_sliding(buffer, chunk, load_data_ML)
        output = buffer.T
        
        if output[0][0] > -10000:
            print(output)
            print(output.shape)
            format_output = np.array([output])
            print(format_output.shape)
            time.sleep(4)
        
    print('Streams closed')

if __name__ == "__main__":
    main()
    

looking for an EEG stream...
[[ -284 -1000  -673 ...  -624 -1000    47]
 [ -308 -1000  -705 ...  -616 -1000   -43]
 [  609  -927 -1000 ...  -411 -1000  -615]
 [ -588 -1000  -313 ...  -918 -1000   479]]
(4, 750)
(1, 4, 750)
[[-1000  -673   897 ... -1000    47  -933]
 [-1000  -705   863 ... -1000   -43  -958]
 [ -927 -1000   662 ... -1000  -615   770]
 [-1000  -313   900 ... -1000   479   975]]
(4, 750)
(1, 4, 750)
[[ -673   897   506 ...    47  -933  -286]
 [ -705   863   500 ...   -43  -958  -270]
 [-1000   662   812 ...  -615   770   700]
 [ -313   900  -339 ...   479   975  -590]]
(4, 750)
(1, 4, 750)
[[  897   506 -1000 ...  -933  -286 -1000]
 [  863   500 -1000 ...  -958  -270 -1000]
 [  662   812  -814 ...   770   700  -901]
 [  900  -339 -1000 ...   975  -590 -1000]]
(4, 750)
(1, 4, 750)
[[  506 -1000  -958 ...  -286 -1000  -747]
 [  500 -1000  -953 ...  -270 -1000  -733]
 [  812  -814 -1000 ...   700  -901 -1000]
 [ -339 -1000  -800 ...  -590 -1000  -270]]
(4, 750)
(1, 4, 750)
[

KeyboardInterrupt: 

In [ ]:
import threading, time

class myThread(threading.Thread):
       def __init__(self,name):
               threading.Thread.__init__(self)
               self.name=name
       def run(self):
               lock.acquire()
               run_model(self.name)
               lock.release()

def run_model(threadName):
    print(threadName,"has acquired lock and is running model")
    time.sleep(5)
    print('\nlock released for', threadName)


    
            